In [5]:
# !pip install langchain langchain_openai chromadb streamlit wikipedia
# !pip install langchain_community

In [6]:
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
def load_docs(query):
    loader = WikipediaLoader(query=query, load_max_docs=1)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [ ]:
def create_vectorstore(splits):
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    return vectorstore

In [ ]:
def create_rag_chain(vectorstore):
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    
    prompt_template = """아래의 문맥을 사용하여 질문에 답하십시오.
    만약 답을 모른다면, 모른다고 말하고 답을 지어내지 마십시오.
    최대한 세 문장으로 답하고 가능한 한 간결하게 유지하십시오.
    {context}
    질문: {question}
    유용한 답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [ ]:
import streamlit as st
from rag_functions import load_docs, create_vectorstore, create_rag_chain

st.title("RAG Q&A 시스템")

# 사용자 입력
topic = st.text_input("위키피디아 주제를 입력하세요:")
question = st.text_input("해당 주제에 대해 질문하세요:")

if topic and question:
    if st.button("답변 받기"):
        with st.spinner("처리 중..."):
            # 문서 로드 및 분할
            splits = load_docs(topic)
            
            # 벡터 저장소 생성
            vectorstore = create_vectorstore(splits)
            
            # RAG 체인 생성
            qa_chain = create_rag_chain(vectorstore)
            
            # 질문에 대한 답변 생성
            result = qa_chain({"query": question})
            
            st.subheader("답변:")
            st.write(result["result"])
            
            st.subheader("출처:")
            for doc in result["source_documents"]:
                st.write(doc.page_content)
                st.write("---")

st.sidebar.title("소개")
st.sidebar.info(
    "이 앱은 RAG(검색 증강 생성) 시스템을 시연합니다. "
    "위키피디아를 지식 소스로 사용하고 OpenAI의 GPT 모델을 통해 답변을 생성합니다."
)